### Chat Implementation with File Chat Message history

In [3]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import FileChatMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec

# --- 1. Setup ---
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")


FILE_PATH = "persistent_chat_log.json"
SESSION_ID = "main-user-session"

# --- 2. Define the LLM and Chain (The Logic) ---

# Define the Prompt with the critical MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful and friendly assistant named Geo. Keep your answers concise and conversational."),
        # This is where the chat history from the FileChatMessageHistory will be injected
        MessagesPlaceholder(variable_name="chat_history"),
        # This is where the current user input will be injected
        ("human", "{input}"),
    ]
)

# Define the LLM and the Core Chain (Prompt | LLM | Parser)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3)
chain = prompt | llm | StrOutputParser()

# --- 3. Define the History Manager Function ---

# This function tells RunnableWithMessageHistory how to get the correct history object
def get_session_history(session_id: str) -> FileChatMessageHistory:
    """Uses the session ID to create a unique FileChatMessageHistory object."""
    # We use a static file name here, but in a real app, you might use f"{session_id}.json"
    return FileChatMessageHistory(file_path=FILE_PATH)

# --- 4. Wrap the Chain with History ---

# RunnableWithMessageHistory wraps your chain and manages the chat_history variable
chain_with_history = RunnableWithMessageHistory(
    runnable=chain,
    get_session_history=get_session_history,
    # Maps the key "input" from the user to the prompt variable {input}
    input_messages_key="input", 
    # Maps the key "chat_history" from the history function to the MessagesPlaceholder
    history_messages_key="chat_history",
    # Define a configurable field spec to ensure the session_id is always present
    # We use a fixed ID here for simplicity
    configurable={
        "session_id": ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            default=SESSION_ID,
            is_shared=True,
        )
    }
)

# --- 5. The Chat Loop (Getting User Input) ---

print("Chat started (type 'quit' or 'exit' to end the session).")
print("-" * 30)

while True:
    try:
        user_input = input("You: ")
        
        # Check for exit command
        if user_input.lower() in ["quit", "exit"]:
            print("\n Session ended. Your history is saved to:", FILE_PATH)
            break
            
        # Invoke the chain with the user input and the session config
        response = chain_with_history.invoke(
            {"input": user_input},
            # Pass the session_id so the history manager knows which file to use
            config={"configurable": {"session_id": SESSION_ID}}
        )
        
        print(f"Geo: {response}")

    except KeyboardInterrupt:
        print("\n Session ended by user (Ctrl+C). Your history is saved to:", FILE_PATH)
        break
    except Exception as e:
        print(f"\nAn error occurred: {e}")
        break

Chat started (type 'quit' or 'exit' to end the session).
------------------------------


You:  what is langchain


Geo: Langchain is a framework designed to simplify the creation of applications using large language models (LLMs). It provides tools and abstractions that help you connect LLMs to other data sources and build more complex and useful AI applications.


You:  explain its ecosystem


Geo: Okay, so the Langchain ecosystem is made up of several key parts that all work together:

*   **LangChain Libraries:** These are the core Python and JavaScript libraries with modules for models, prompts, chains, data connection, agents, and memory.
*   **LangChain Templates:** A collection of easily deployable reference architectures for a variety of tasks.
*   **LangChain Hub:** A central place to discover and share prompts, chains, and agents.
*   **Integrations:** LangChain supports integrations with numerous third-party services like vector databases, APIs, and other tools.
*   **Community:** A large and active community contributing to the project through code, documentation, and support.

In short, it's a comprehensive set of tools, integrations, and resources designed to make building LLM-powered applications easier and more efficient.


...

Geo: Okay, let's talk Langchain memory! Memory helps agents "remember" previous interactions. Here are some common types:

*   **ConversationBufferMemory:** Simplest form, it stores the entire conversation history.
*   **ConversationBufferWindowMemory:** Stores a rolling window of the most recent interactions.
*   **ConversationSummaryMemory:** Summarizes the conversation over time to keep the memory concise.
*   **ConversationSummaryBufferMemory:** Combines the buffer and summary approaches. It uses a buffer initially, then summarizes when the buffer gets too large.
*   **ConversationKnowledgeGraphMemory:** Uses a knowledge graph to store and retrieve information from the conversation.

The best type of memory depends on the specific application and how much context the agent needs to retain!


You:  what is map reduce and runnable


Geo: Okay, let's clarify Map Reduce and Runnable in the context of Langchain:

*   **Map Reduce:** In Langchain, Map Reduce is a type of chain often used for processing large documents. It works by:
    *   **Mapping:** Splitting the document into chunks and processing each chunk independently with an LLM.
    *   **Reducing:** Combining the results from each chunk into a single, final output using another LLM call. This is great for summarizing long texts or answering questions based on a large corpus of data.

*   **Runnable:** In Langchain, a "Runnable" is a standard interface that represents a callable unit. It's a core concept that allows you to easily chain together different components like models, prompts, and other runnables. Anything that implements the Runnable interface can be invoked, composed, and streamed, making it super flexible for building complex workflows.

In short, Map Reduce is a specific type of chain for processing large amounts of data, while Runnable is a mo

You:  exit



 Session ended. Your history is saved to: persistent_chat_log.json
